In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import transforms3d as tf3d
import matplotlib
import scipy.signal
import time
from numba import jit
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import os


In [2]:
n_propellers=6


mass=369 #batterie
mass+=1640-114 #corps-carton
mass/=1e3
Area=np.pi*(11.0e-02)**2
r0=11e-02
rho0=1.204
kv_motor=800.0
pwmmin=1075.0
pwmmax=1950.0
U_batt=16.8

AN=n_propellers*Area
b10=14.44

In [3]:
# %%   ####### IMPORT DATA 
print("LOADING DATA...")


log_path="./logs/target/log_real_processed.csv"

raw_data=pd.read_csv(log_path)

print("PROCESSING DATA...")


prep_data=raw_data.drop(columns=[i for i in raw_data.keys() if (("forces" in i ) or ('pos' in i) or ("joy" in i)) ])
prep_data=prep_data.drop(columns=[i for i in raw_data.keys() if (("level" in i ) or ('Unnamed' in i) or ("index" in i)) ])
# print(prep_data)

tmin,tmax=-1,1e10

    
prep_data=prep_data[prep_data['t']>tmin]
prep_data=prep_data[prep_data['t']<tmax]
prep_data=prep_data.reset_index()
for i in range(3):
    prep_data['speed_pred[%i]'%(i)]=np.r_[prep_data['speed[%i]'%(i)].values[1:len(prep_data)],0]
    
    

prep_data['dt']=np.r_[prep_data['t'].values[1:]-prep_data['t'].values[:-1],0]
prep_data['t']-=prep_data['t'][0]
prep_data=prep_data.drop(index=[0,len(prep_data)-1])


for i in range(n_propellers):
    prep_data['omega_c[%i]'%(i+1)]=(prep_data['PWM_motor[%i]'%(i+1)]-pwmmin)/(pwmmax-pwmmin)*U_batt*kv_motor*2*np.pi/60

R_array=np.array([tf3d.quaternions.quat2mat([i,j,k,l]) for i,j,k,l in zip(prep_data['q[0]'],
                                                                          prep_data['q[1]'],
                                                                          prep_data['q[2]'],
                                                                          prep_data['q[3]'])])

vwx_body_dir=np.array([i.T@np.array([[1.0],[0],[0]]) for i in R_array] )
vwy_body_dir=np.array([i.T@np.array([[0.0],[1.0],[0]]) for i in R_array] )


prep_data['vwx_body_dir']=[i for i in vwx_body_dir.reshape((-1,3))]
prep_data['vwy_body_dir']=[i for i in vwy_body_dir.reshape((-1,3))]
prep_data['vwx_body_dir']=[i for i in vwx_body_dir.reshape((-1,3))]
prep_data['R']=[i for i in R_array]

LOADING DATA...
PROCESSING DATA...


In [4]:
# %% Global Optimization ACC

def compute_single_motor_thrust_(omega,vai,vaj,vak,c1,c2=0,c3=0,vanilla_test=False):
    
    if vanilla_test:
        T=rho0*Area*r0**2*omega**2*c1
        return T
    else:
        eta=vak/2-r0*omega*c2/4
        eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
                            +2*c1*r0**2*omega**2+2*c3*(vai**2+vaj**2))

        T=rho0*Area*r0*omega*(c1*r0*omega-c2*(eta-vak))+rho0*Area*c3*(vai**2+vaj**2)
        return T

def compute_omegas(df,kt):
    # omegas is shape n timesteps x p actuators 
    omegas=np.array([df['omega_c[%i]'%(i+1)] for i in range(n_propellers)]).T
    
    b, a = scipy.signal.butter(1, 1./(2*np.pi)*kt,analog= False,fs=1./2.0/df['dt'].mean())
    zi = scipy.signal.lfilter_zi(b, a)
    new_omegas, _ =scipy.signal.lfilter(b, a, omegas,axis=0,zi=zi*omegas[0].reshape((1,-1)))
    return new_omegas


def compute_H(omega,vai,vaj,vak,ch1,ch2,c1=0,c2=0,c3=0,vanilla_test=False):
    if vanilla_test:
        return np.zeros(omega.shape)
    else:
        eta=vak/2-r0*omega*c2/4
        eta=eta+0.5*np.sqrt((vak+0.5*r0*omega*c2)**2
                            +2*c1*r0**2*omega**2+2*c3*(vai**2+vaj**2))
        H=rho0*Area*(ch1*r0*omega-ch2*(eta-vak))
        return H

def compute_acc_identification_globale(ct1,
                                       ct2,
                                       ch1,
                                       ch2,
                                       di=0,
                                       dj=0,
                                       dk=0,
                                       df=prep_data,
                                       kt=0.0,
                                       ct3=0,
                                       vwi=0,
                                       vwj=0,
                                      vanilla=False,
                                       dyn_motors=False):
    
    vw_body=vwi*np.array([i for i in df['vwx_body_dir'].values])+vwj*np.array([i for i in df['vwy_body_dir'].values])
    
    
    vai=df["speed_body[0]"]-vw_body[:,0]
    vaj=df["speed_body[1]"]-vw_body[:,1]
    vak=df["speed_body[2]"]-vw_body[:,2]
    
    gammai=df["gamma[0]"]
    gammaj=df["gamma[1]"]
    gammak=df["gamma[2]"]
    
    omegas = compute_omegas(df,kt).reshape((len(df),6)) if dyn_motors else np.array([df['omega_c[%i]'%(i+1)] for i in range(n_propellers)]).T
    
    
    T=sum([compute_single_motor_thrust_(omegas[:,i],
         vai,vaj,vak,
                    ct1,ct2,ct3,vanilla_test=vanilla) for i in range(n_propellers)])
    
    H=sum([compute_H(omegas[:,i],
          vai,vaj,vak,ch1,ch2,
                     ct1,ct2,ct3,vanilla_test=vanilla) for i in range(n_propellers)])
    
    H_vect=np.c_[-vai*H,-vaj*H,np.zeros(H.shape)]
    T_vect=np.c_[np.zeros(T.shape),np.zeros(T.shape),T]
    
    absva=np.sqrt(vai**2+vaj**2+vak**2)
    Fa=-rho0*AN*np.c_[di*absva*vai,dj*absva*vaj,dk*absva*vak]

    return -T_vect/mass+H_vect/mass+np.c_[gammai,gammaj,gammak]+Fa/mass


def cost_global_(X,option_booleans,rms_score=False):
    
    ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj = X

    motor_dyn , vanilla , with_c3 , dij , with_wind = option_booleans

    ct3 = ct3 if with_c3 else 0
    vwi,vwj=(vwi,vwj) if with_wind else (0,0) 
    dj = di if dij else dj
          
          
    Y=compute_acc_identification_globale(ct1,
                                       ct2,
                                       ch1,
                                       ch2,                                       
                                        di=di,
                                       dj=dj,
                                       dk=dk,
                                       kt=kt,
                                       ct3=ct3,
                                       vwi=vwi,
                                       vwj=vwj,
                                      vanilla=vanilla,
                                       dyn_motors=motor_dyn)
          
    ci=np.mean((Y[:,0]-prep_data['acc_body_grad[0]'])**2,axis=0)
    cj=np.mean((Y[:,1]-prep_data['acc_body_grad[1]'])**2,axis=0)
    ck=np.mean((Y[:,2]-prep_data['acc_body_grad[2]'])**2,axis=0)
    if rms_score==False:
        ci/=max(abs(prep_data['acc_body_grad[0]']))**2
        cj/=max(abs(prep_data['acc_body_grad[1]']))**2
        ck/=max(abs(prep_data['acc_body_grad[2]']))**2
        
        
        c=ci+cj+ck
#         print("ct1:%f ct2:%f ch1:%f ch2:%f di:%f dj:%f dk:%f ct3:%f kt:%f vwi:%f vwj:%f cost:%f"%(ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj,c))

        return c

    else:
        score_rms_total=np.mean(np.linalg.norm(Y-np.c_[prep_data['acc_body_grad[0]'],prep_data['acc_body_grad[1]'],prep_data['acc_body_grad[2]']],axis=1))
        return np.sqrt([score_rms_total,ci,cj,ck])

    
    
    

In [5]:

params = []

for motor_dyn in (True,False):
    for vanilla in (True,False):
        for with_c3 in (True,False):
            for dij in (True,False):
                for with_wind in (True,False):
                    if not( vanilla and with_c3):
                        params.append([motor_dyn , vanilla , with_c3 , dij , with_wind])  
                    
print("N combinaisons: ",len(params))

keys_dict_=['ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj'.split(',')]


X0_id_globale=np.array([0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0,
                        0, #c3
                        5.0, #kt
                        0, #vwi
                        0]) #vwj



headers_df = ['motor_dyn,vanilla,with_c3,dij,with_wind'.replace(' ','').split(',')]
headers_df = headers_df + ['ct1,ct2,ch1,ch2,di,dj,dk,ct3,kt,vwi,vwj'.replace(' ','').split(',')]
headers_df = headers_df + ['score_opti,score_rms,score_x,score_y,score_z,duration'.replace(' ','').split(',')]
headers_df = [item for sublist in headers_df for item in sublist]


N combinaisons:  24


In [6]:
try:
    os.makedirs(os.path.join("results","acc"))
except:
    pass

In [7]:
# %% global acc opti: multi thread run
from multiprocessing import Pool
import time

def run_in_process(params_el):
    
    acc_data_list=[]

    t0=time.time()
    
    sol_= minimize(cost_global_,X0_id_globale,args=(params_el))
    score_rms,score_x,score_y,score_z = cost_global_(sol_['x'],params_el,rms_score=True)
    
    optim_duration = time.time()-t0
    
    acc_data_list.append(np.r_[params_el,sol_['x'],sol_['fun'],score_rms,score_x,score_y,score_z,optim_duration])
    
    df_res =  pd.DataFrame(data=acc_data_list,columns=headers_df)   
    sname=''
    for i in params_el:
        sname=sname+'T' if i else sname+'F' 
    df_res.to_csv('./results/acc/acc_'+sname+".csv")
    print(params_el,"Done...")
    return True
    
if __name__ == '__main__':

    pool = Pool(processes=6)
    pool.map(run_in_process, params)

    

/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarn

[True, False, True, True, False] Done...


/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


[True, False, True, True, True] Done...


/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


...

[False, False, True, False, True] Done...


/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/alex/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


[False, False, True, False, False] Done...
[False, False, False, True, True] Done...
[False, False, False, True, False] Done...
[False, False, False, False, False] Done...
[False, False, False, False, True] Done...


ValueError: Digital filter critical frequencies must be 0 < Wn < fs/2 (fs=118.37590876641497 -> fs/2=59.18795438320748)

Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
